In [10]:
import hashlib
import time
from collections import defaultdict
from ecdsa import SigningKey, NIST384p

class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_string = f"{self.index}{self.transactions}{self.timestamp}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        self.chain.append(genesis_block)

    def add_new_transaction(self, transaction):
        self.current_transactions.append(transaction)

    def add_block(self, block):
        previous_hash = self.chain[-1].hash
        block.previous_hash = previous_hash
        block.hash = block.compute_hash()
        self.chain.append(block)
        self.current_transactions = []

    def create_new_block(self):
        block = Block(len(self.chain), self.current_transactions, time.time(), self.chain[-1].hash)
        self.add_block(block)

class Wallet:
    def __init__(self):
        self.private_key = SigningKey.generate(curve=NIST384p)
        self.public_key = self.private_key.get_verifying_key()

    def create_transaction(self, sender_public_key, recipient_public_key, amount, coin_type):
        transaction = {
            'sender': sender_public_key.to_string().hex(),
            'recipient': recipient_public_key.to_string().hex(),
            'amount': amount,
            'coin_type': coin_type
        }
        transaction['signature'] = self.private_key.sign(str(transaction).encode()).hex()
        return transaction

# Example Usage
blockchain = Blockchain()
wallet1 = Wallet()
wallet2 = Wallet()

transaction1 = wallet1.create_transaction(wallet1.public_key, wallet2.public_key, 10, 'CoinA')
blockchain.add_new_transaction(transaction1)

blockchain.create_new_block()

print(f"Blockchain: {[block.__dict__ for block in blockchain.chain]}")


ModuleNotFoundError: No module named 'ecdsa'